In [1]:
import datetime
import json
import requests
import time
import pandas as pd

In [2]:
keys = json.loads(open("keys.json").read())

# Twitch

In [3]:
twitch_client_ID = keys['Twitch']['Client-ID']

In [4]:
header = {'Client-ID': twitch_client_ID}

## Previous Functions

In [5]:
def print_response(response, start = 0):
    '''
    prints the response in a more readable way.
    'start' specifies the first number of the list
    '''
    for i in range(len(response.json()['data'])):
        print('{:2d} {:6s} :  {}'.format(i + 1 + start, 
                                      response.json()['data'][i]['id'],
                                      response.json()['data'][i]['name']))

In [6]:
def print_games_list(l):
    '''
    l must be a list of games dicts, with keys "name" and "id".
    the function prints l.
    '''
    for i in range(len(l)):
            print('{:2d} {:6s} :  {}'.format(i + 1, l[i]['id'], l[i]['name']))            

In [7]:
def res_pag(response):
    '''
    returns pagination cursor of a response
    '''
    return response.json()['pagination']['cursor']

In [8]:
# old version with less control on numer of games

def get_n_top_games_OLD(n, header):
    '''
    returns the ordered list of top games (dicts), with length between n and (n + 99)
    headers is the header of the request (specifying the Client-ID)
    '''
    responses = [] # list of games
    
    res = requests.get('https://api.twitch.tv/helix/games/top?first=100', headers=header)
    responses.extend(res.json()['data'])
           
    while len(responses) < n:
        res = requests.get('https://api.twitch.tv/helix/games/top?after={}&first=100'.format(res_pag(res)), headers=header)
        responses.extend(res.json()['data'])
    
    return responses

In [9]:
# I wrote the try - except thing in case errors occur (e.g. 'too many requests'), in which case 
# 'res.json()['data'] does not exist

def get_views(game_id, n = 100):
    '''
    returns total number of view for game which id is game_id, for the top n <= 100 streams
    '''
    res = requests.get('https://api.twitch.tv/helix/streams?game_id={}&first={}'.format(game_id, n), headers=header)
    try:
        df = pd.DataFrame(res.json()['data'])
        s = sum(df['viewer_count'])
        #s = sum([res.json()['data'][i]['viewer_count'] for i in range(len(res.json()['data']))])
    except: 
        s = res.json()
    return s, res

In [10]:
# newer version, possibly we could make it better by making it wait if we surpass the 30 requests per minute limit
# for some reason if n <= 15 it returns one more game

def get_n_top_games(n, header, exactly_n = False):
    '''
    returns the ordered list of top n-1 games (dicts)
    headers is the header of the request (specifying the Client-ID)
    if exactly_n == True, the function returns the list of top n games
        by default this is false as it performs one additional request just to get a single game
    '''
    responses = [] # list of games
    
    m = n // 100
    q = n % 100
        
    if m == 0:
        # if n is less than or equal to 100, just gets the first n games
        res = requests.get(
            'https://api.twitch.tv/helix/games/top?first={}'.format(n),
            headers=header)
        responses.extend(res.json()['data'])
    else:
        # get the first 99 games:
        res = requests.get(
            'https://api.twitch.tv/helix/games/top?first=100',
            headers=header)
        responses.extend(res.json()['data']) 

        # loop to get the remaining (m-1)*100 games
        m -= 1
        while m > 0:
            res = requests.get(
                'https://api.twitch.tv/helix/games/top?after={}&first=100'.format(res_pag(res)),
                headers=header)
            responses.extend(res.json()['data'])
            m -= 1

        # now m = 0, we get the last q games if q > 0
        if q > 0:
            res = requests.get(
                'https://api.twitch.tv/helix/games/top?after={}&first={}'.format(res_pag(res), q),
                headers=header)
            responses.extend(res.json()['data'])

    if exactly_n:
        # get the last game
        res = requests.get(
            'https://api.twitch.tv/helix/games/top?after={}&first={}'.format(res_pag(res), 1),
            headers=header)
        responses.extend(res.json()['data'])
    
    return responses

In [11]:
# I wrote the try - except thing in case errors occur (e.g. 'too many requests'), in which case 
# 'res.json()['data'] does not exist

def get_views(game_id, n = 100):
    '''
    returns total number of view for game which id is game_id, for the top n <= 100 streams
    '''
    res = requests.get('https://api.twitch.tv/helix/streams?game_id={}&first={}'.format(game_id, n), headers=header)
    try:
        df = pd.DataFrame(res.json()['data'])
        s = sum(df['viewer_count'])
        #s = sum([res.json()['data'][i]['viewer_count'] for i in range(len(res.json()['data']))])
    except: 
        s = res.json()
    return s, res

In [12]:
# If we make too many requests (the limit is 30 per minute), the function waits a minute

def integrate_view_count(game_list, print_progress = False):
    '''
    integrates the list outputted by the function get_n_top_games() with the number of views for each game
    if print_progress = True it prints the progress
    '''
    len_game_list = len(game_list)
    for i in range(len_game_list):
        res = get_views(game_list[i]['id'], 100)
        game_list[i]['view_count'] = res[0]
        
        games_left = len_game_list - i - 1
        limit = int(res[1].headers['Ratelimit-Remaining'])
        if print_progress: print('Games left: {}\tPoints left: {}'.format(games_left, limit))
        
        if (limit == 0) and (games_left != 0):
            if print_progress: print('Processed {} games.\nHave to wait a minute...'.format(i+1))
            for j in range(60):
                if print_progress: print(60 - j)
                time.sleep(1)            

## Get all games (new APIs)

In [13]:
def get_all_top_games(header, print_progress = False):
    '''
    returns list of all games streamed on twitch, sorted by viewers
    if print_progress = True it prints the progress
    '''
    responses = [] # list of games
    
    # first request 
    res = requests.get(
            'https://api.twitch.tv/helix/games/top?first=100',
            headers=header)
    responses.extend(res.json()['data']) 
    
    games_processed = len(res.json()['data'])
    
    # ask for games until there are no games left
    while len(res.json()['data']) > 0:
        res = requests.get(
                'https://api.twitch.tv/helix/games/top?after={}&first=100'
            .format(res_pag(res)),
                headers=header)
        responses.extend(res.json()['data'])
        
        games_processed += len(res.json()['data'])
        limit = int(res.headers['Ratelimit-Remaining'])
        
        if print_progress:
            print('games_processed: {}, remaining points = {}'.format(games_processed, limit))
             
        if limit == 0:
            if print_progress: print('Have to wait a minute...')
            for j in range(60):
                if print_progress: print(60 - j)
                time.sleep(1)
    
    if print_progress: print('Done!')
    return responses

In [14]:
all_top_games = get_all_top_games(header, print_progress = True)

games_processed: 199, remaining points = 28
games_processed: 299, remaining points = 27
games_processed: 399, remaining points = 26
games_processed: 499, remaining points = 25
games_processed: 599, remaining points = 24
games_processed: 699, remaining points = 23
games_processed: 799, remaining points = 22
games_processed: 899, remaining points = 22
games_processed: 999, remaining points = 21
games_processed: 1099, remaining points = 20
games_processed: 1199, remaining points = 19
games_processed: 1299, remaining points = 18
games_processed: 1399, remaining points = 17
games_processed: 1499, remaining points = 16
games_processed: 1599, remaining points = 16
games_processed: 1699, remaining points = 15
games_processed: 1797, remaining points = 14
games_processed: 1842, remaining points = 13
games_processed: 1842, remaining points = 12
Done!


In [15]:
atgdf = pd.DataFrame(all_top_games)

In [16]:
pd.options.display.max_colwidth = 120 # to display the complete box_art_url column

In [17]:
atgdf.head()

,box_art_url,id,name
0,https://static-cdn.jtvnw.net/ttv-boxart/League%20of%20Legends-{width}x{height}.jpg,21779,League of Legends
1,https://static-cdn.jtvnw.net/ttv-boxart/Dota%202-{width}x{height}.jpg,29595,Dota 2
2,https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-{width}x{height}.jpg,33214,Fortnite
3,https://static-cdn.jtvnw.net/ttv-boxart/TrackMania%C2%B2%20Stadium-{width}x{height}.jpg,79231,TrackMania² Stadium
4,https://static-cdn.jtvnw.net/ttv-boxart/Just%20Chatting-{width}x{height}.jpg,509658,Just Chatting


In [18]:
atgdf.index += 1 # to make index = order of games by views

In [19]:
atgdf.head()

,box_art_url,id,name
1,https://static-cdn.jtvnw.net/ttv-boxart/League%20of%20Legends-{width}x{height}.jpg,21779,League of Legends
2,https://static-cdn.jtvnw.net/ttv-boxart/Dota%202-{width}x{height}.jpg,29595,Dota 2
3,https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-{width}x{height}.jpg,33214,Fortnite
4,https://static-cdn.jtvnw.net/ttv-boxart/TrackMania%C2%B2%20Stadium-{width}x{height}.jpg,79231,TrackMania² Stadium
5,https://static-cdn.jtvnw.net/ttv-boxart/Just%20Chatting-{width}x{height}.jpg,509658,Just Chatting


In case we ever need the box art for the games (maybe for some part of visualization?), we get the full url with the following (width and height are the ones used by twitch)

In [20]:
atgdf['box_art_url'] = atgdf['box_art_url'].apply(lambda x : x.format(width = 285, height = 380))

In [21]:
atgdf.head()

,box_art_url,id,name
1,https://static-cdn.jtvnw.net/ttv-boxart/League%20of%20Legends-285x380.jpg,21779,League of Legends
2,https://static-cdn.jtvnw.net/ttv-boxart/Dota%202-285x380.jpg,29595,Dota 2
3,https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-285x380.jpg,33214,Fortnite
4,https://static-cdn.jtvnw.net/ttv-boxart/TrackMania%C2%B2%20Stadium-285x380.jpg,79231,TrackMania² Stadium
5,https://static-cdn.jtvnw.net/ttv-boxart/Just%20Chatting-285x380.jpg,509658,Just Chatting


Looking for duplicate rows

In [22]:
atgdf.sort_values('name', inplace = True) # create a copy of the df, sorted by name

In [23]:
atgdf.head()

,box_art_url,id,name
1398,https://static-cdn.jtvnw.net/ttv-boxart/.hack//G.U.%20Last%20Recode-285x380.jpg,497460,.hack//G.U. Last Recode
942,https://static-cdn.jtvnw.net/ttv-boxart/./007:%20Nightfire-285x380.jpg,6081,007: Nightfire
1713,https://static-cdn.jtvnw.net/ttv-boxart/./24:%20The%20Game-285x380.jpg,12580,24: The Game
1383,https://static-cdn.jtvnw.net/ttv-boxart/60%20Parsecs%21-285x380.jpg,502317,60 Parsecs!
1268,https://static-cdn.jtvnw.net/ttv-boxart/60%20Parsecs%21-285x380.jpg,502317,60 Parsecs!


In [24]:
# look for duplicates 
# (checks if two following rows have the same name:
# if so, appends the df index of the row to the first list.
# and the implicit index of the sorted df to the second list)

duplicates = list()
duplicates_implicit = list()
for i in range(len(atgdf)-1):
    if atgdf['name'].iloc[i] == atgdf['name'].iloc[i+1]:
        duplicates.append(atgdf.iloc[i].name)
        duplicates_implicit.append(i)
len(duplicates), len(duplicates_implicit)

(262, 262)

In [25]:
atgdf.loc[duplicates]

,box_art_url,id,name
1383,https://static-cdn.jtvnw.net/ttv-boxart/60%20Parsecs%21-285x380.jpg,502317,60 Parsecs!
1697,https://static-cdn.jtvnw.net/ttv-boxart/9-1-1%20Operator-285x380.jpg,493317,9-1-1 Operator
1728,https://static-cdn.jtvnw.net/ttv-boxart/A%20Story%20About%20My%20Uncle-285x380.jpg,67895,A Story About My Uncle
1009,https://static-cdn.jtvnw.net/ttv-boxart/Absolver-285x380.jpg,494591,Absolver
1434,https://static-cdn.jtvnw.net/ttv-boxart/Action-285x380.jpg,25749,Action
1618,https://static-cdn.jtvnw.net/ttv-boxart/./Age%20of%20Conan:%20Unchained-285x380.jpg,8841,Age of Conan: Unchained
1422,https://static-cdn.jtvnw.net/ttv-boxart/Age%20of%20Empires%20III-285x380.jpg,7830,Age of Empires III
1296,https://static-cdn.jtvnw.net/ttv-boxart/Age%20of%20Empires%20III-285x380.jpg,7830,Age of Empires III
697,https://static-cdn.jtvnw.net/ttv-boxart/Alone%20in%20the%20Dark-285x380.jpg,2846,Alone in the Dark
650,https://static-cdn.jtvnw.net/ttv-boxart/Ape%20Escape%203-285x380.jpg,17747,Ape Escape 3


In [26]:
atgdf.sort_index(inplace = True) # go back to order by views

In [27]:
atgdf.head()

,box_art_url,id,name
1,https://static-cdn.jtvnw.net/ttv-boxart/League%20of%20Legends-285x380.jpg,21779,League of Legends
2,https://static-cdn.jtvnw.net/ttv-boxart/Dota%202-285x380.jpg,29595,Dota 2
3,https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-285x380.jpg,33214,Fortnite
4,https://static-cdn.jtvnw.net/ttv-boxart/TrackMania%C2%B2%20Stadium-285x380.jpg,79231,TrackMania² Stadium
5,https://static-cdn.jtvnw.net/ttv-boxart/Just%20Chatting-285x380.jpg,509658,Just Chatting


## Get all games (API v5)

In [28]:
header_v5 = {
    'Accept': 'application/vnd.twitchtv.v5+json',
    'Client-ID': twitch_client_ID,
}

In [29]:
def print_games_list_v5(l):
    '''
    l must be a list of games dicts, with keys "name" and "id".
    the function prints l.
    '''
    for i in range(len(l)):
            print('{:2d} {:6d} :  {:40s} {} '.format(i + 1, l[i]['game']['_id'], l[i]['game']['name'], l[i]['viewers']))

In [30]:
def get_all_top_games_v5(header, print_progress = False):
    '''
    returns list of all games streamed on twitch, sorted by viewers
    if print_progress = True it prints the progress
    '''
    responses = [] # list of games
    
    # first request 
    res = requests.get('https://api.twitch.tv/kraken/games/top?limit=100', headers=header)
    responses.extend(res.json()['top'])
    
    games_processed = len(res.json()['top'])
    
    # ask for games until there are no games left
    
    # ask for games until there are no games left (in which case, the twitch api returns an error)
    # essentially, it loops requests until twitch returns an error, in which case the loops breaks
    # not the best solution, but I do not know if there is a better way to do this
    while True:
        try:
            res = requests.get('https://api.twitch.tv/kraken/games/top?limit=100&offset={}'.format(len(responses)), headers=header)
            responses.extend(res.json()['top'])
            
            games_processed += len(res.json()['top'])
        
            if print_progress:
                print('games_processed: {}'.format(games_processed)) 
            
        except Exception:
            if print_progress: print('Done!')
            break
            
    return responses

In [31]:
v5_all = get_all_top_games_v5(header_v5, print_progress = True)

games_processed: 199
games_processed: 299
games_processed: 399
games_processed: 499
games_processed: 599
games_processed: 699
games_processed: 799
games_processed: 899
games_processed: 999
games_processed: 1099
games_processed: 1199
games_processed: 1299
games_processed: 1399
games_processed: 1498
games_processed: 1597
games_processed: 1696
games_processed: 1795
games_processed: 1834
games_processed: 1845
games_processed: 1846
Done!


In [32]:
v5_all_df = pd.DataFrame(v5_all)

In [33]:
v5_all_df.head() 

,channels,game,viewers
0,2244,"{'name': 'League of Legends', 'popularity': 451586, '_id': 21779, 'giantbomb_id': 24024, 'box': {'large': 'https://s...",384312
1,857,"{'name': 'Dota 2', 'popularity': 188746, '_id': 29595, 'giantbomb_id': 32887, 'box': {'large': 'https://static-cdn.j...",213073
2,8743,"{'name': 'Fortnite', 'popularity': 88708, '_id': 33214, 'giantbomb_id': 37030, 'box': {'large': 'https://static-cdn....",93392
3,27,"{'name': 'TrackMania² Stadium', 'popularity': 49055, '_id': 79231, 'giantbomb_id': 40347, 'box': {'large': 'https://...",84775
4,1099,"{'name': 'Just Chatting', 'popularity': 62125, '_id': 509658, 'giantbomb_id': 0, 'box': {'large': 'https://static-cd...",76581


In [34]:
from pandas.io.json import json_normalize

In [35]:
# to unnest the game info

v5_all_df = json_normalize(data=v5_all)
v5_all_df.index += 1
v5_all_df = v5_all_df[['game._id', 'game.name', 'game.localized_name', 'channels', 'viewers', 'game.popularity', 'game.box.large', 'game.logo.large']]

In [36]:
v5_all_df.head()

,game._id,game.name,game.localized_name,channels,viewers,game.popularity,game.box.large,game.logo.large
1,21779,League of Legends,League of Legends,2244,384312,451586,https://static-cdn.jtvnw.net/ttv-boxart/League%20of%20Legends-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/League%20of%20Legends-240x144.jpg
2,29595,Dota 2,Dota 2,857,213073,188746,https://static-cdn.jtvnw.net/ttv-boxart/Dota%202-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Dota%202-240x144.jpg
3,33214,Fortnite,Fortnite,8743,93392,88708,https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Fortnite-240x144.jpg
4,79231,TrackMania² Stadium,TrackMania² Stadium,27,84775,49055,https://static-cdn.jtvnw.net/ttv-boxart/TrackMania%C2%B2%20Stadium-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/TrackMania%C2%B2%20Stadium-240x144.jpg
5,509658,Just Chatting,Just Chatting,1099,76581,62125,https://static-cdn.jtvnw.net/ttv-boxart/Just%20Chatting-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Just%20Chatting-240x144.jpg


Looking for duplicates again

In [37]:
v5_all_df.sort_values('game.name', inplace=True)

In [38]:
v5_all_df.head()

,game._id,game.name,game.localized_name,channels,viewers,game.popularity,game.box.large,game.logo.large
1366,497460,.hack//G.U. Last Recode,.hack//G.U. Last Recode,1,2,2,https://static-cdn.jtvnw.net/ttv-boxart/.hack//G.U.%20Last%20Recode-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/.hack//G.U.%20Last%20Recode-240x144.jpg
1355,11045,.hack//MUTATION - Part 2,.hack//MUTATION - Part 2,1,2,2,https://static-cdn.jtvnw.net/ttv-boxart/.hack//MUTATION%20-%20Part%202-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/.hack//MUTATION%20-%20Part%202-240x144.jpg
1417,11045,.hack//MUTATION - Part 2,.hack//MUTATION - Part 2,1,2,1,https://static-cdn.jtvnw.net/ttv-boxart/.hack//MUTATION%20-%20Part%202-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/.hack//MUTATION%20-%20Part%202-240x144.jpg
925,6081,007: Nightfire,007: Nightfire,1,6,7,https://static-cdn.jtvnw.net/ttv-boxart/./007:%20Nightfire-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/./007:%20Nightfire-240x144.jpg
1215,502317,60 Parsecs!,60 Parsecs!,1,3,1,https://static-cdn.jtvnw.net/ttv-boxart/60%20Parsecs%21-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/60%20Parsecs%21-240x144.jpg


We get duplicates again.

In [39]:
# look for duplicates 
# (checks if two following rows have the same name: if so, appends the df index of the row to the list)

duplicates_v5 = list()
duplicates_implicit_v5 = list()

for i in range(len(v5_all_df)-1):
    if v5_all_df['game.name'].iloc[i] == v5_all_df['game.name'].iloc[i+1]:
        duplicates_implicit_v5.append(i)
        duplicates_v5.append(v5_all_df.iloc[i].name)

len(duplicates_v5), len(duplicates_implicit_v5)

(251, 251)

In [40]:
v5_all_df.loc[duplicates_v5]

,game._id,game.name,game.localized_name,channels,viewers,game.popularity,game.box.large,game.logo.large
1355,11045,.hack//MUTATION - Part 2,.hack//MUTATION - Part 2,1,2,2,https://static-cdn.jtvnw.net/ttv-boxart/.hack//MUTATION%20-%20Part%202-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/.hack//MUTATION%20-%20Part%202-240x144.jpg
1621,458060,A Bird Story,A Bird Story,1,1,1,https://static-cdn.jtvnw.net/ttv-boxart/A%20Bird%20Story-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/A%20Bird%20Story-240x144.jpg
1033,494591,Absolver,Absolver,2,5,5,https://static-cdn.jtvnw.net/ttv-boxart/Absolver-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Absolver-240x144.jpg
1480,8841,Age of Conan: Unchained,Age of Conan: Unchained,1,1,1,https://static-cdn.jtvnw.net/ttv-boxart/./Age%20of%20Conan:%20Unchained-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/./Age%20of%20Conan:%20Unchained-240x144.jpg
677,349609,America's Army: Proving Grounds,America's Army: Proving Grounds,3,13,9,https://static-cdn.jtvnw.net/ttv-boxart/./America%27s%20Army:%20Proving%20Grounds-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/./America%27s%20Army:%20Proving%20Grounds-240x144.jpg
1778,14192,Ants,Ants,1,0,0,https://static-cdn.jtvnw.net/ttv-boxart/Ants-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Ants-240x144.jpg
1102,511554,Arknights,Arknights,1,4,2,https://static-cdn.jtvnw.net/ttv-boxart/Arknights-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Arknights-240x144.jpg
1460,20988,Ascension to the Throne,Ascension to the Throne,1,1,1,https://static-cdn.jtvnw.net/ttv-boxart/Ascension%20to%20the%20Throne-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Ascension%20to%20the%20Throne-240x144.jpg
1557,20988,Ascension to the Throne,Ascension to the Throne,1,1,1,https://static-cdn.jtvnw.net/ttv-boxart/Ascension%20to%20the%20Throne-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Ascension%20to%20the%20Throne-240x144.jpg
1617,500181,Ashes Cricket,Ashes Cricket,1,1,0,https://static-cdn.jtvnw.net/ttv-boxart/Ashes%20Cricket-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Ashes%20Cricket-240x144.jpg


At least with the old apis we imediately get the viewers.

In [41]:
v5_all_df.sort_index(inplace = True) # go back to the order by viewers

In [42]:
v5_all_df

,game._id,game.name,game.localized_name,channels,viewers,game.popularity,game.box.large,game.logo.large
1,21779,League of Legends,League of Legends,2244,384312,451586,https://static-cdn.jtvnw.net/ttv-boxart/League%20of%20Legends-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/League%20of%20Legends-240x144.jpg
2,29595,Dota 2,Dota 2,857,213073,188746,https://static-cdn.jtvnw.net/ttv-boxart/Dota%202-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Dota%202-240x144.jpg
3,33214,Fortnite,Fortnite,8743,93392,88708,https://static-cdn.jtvnw.net/ttv-boxart/Fortnite-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Fortnite-240x144.jpg
4,79231,TrackMania² Stadium,TrackMania² Stadium,27,84775,49055,https://static-cdn.jtvnw.net/ttv-boxart/TrackMania%C2%B2%20Stadium-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/TrackMania%C2%B2%20Stadium-240x144.jpg
5,509658,Just Chatting,Just Chatting,1099,76581,62125,https://static-cdn.jtvnw.net/ttv-boxart/Just%20Chatting-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Just%20Chatting-240x144.jpg
6,32982,Grand Theft Auto V,Grand Theft Auto V,751,72662,69967,https://static-cdn.jtvnw.net/ttv-boxart/Grand%20Theft%20Auto%20V-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Grand%20Theft%20Auto%20V-240x144.jpg
7,32399,Counter-Strike: Global Offensive,Counter-Strike: Global Offensive,1123,46692,40636,https://static-cdn.jtvnw.net/ttv-boxart/./Counter-Strike:%20Global%20Offensive-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/./Counter-Strike:%20Global%20Offensive-240x144.jpg
8,493057,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS,1365,30651,30977,https://static-cdn.jtvnw.net/ttv-boxart/PLAYERUNKNOWN%27S%20BATTLEGROUNDS-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/PLAYERUNKNOWN%27S%20BATTLEGROUNDS-240x144.jpg
9,138585,Hearthstone,Hearthstone,251,28411,25288,https://static-cdn.jtvnw.net/ttv-boxart/Hearthstone-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Hearthstone-240x144.jpg
10,488552,Overwatch,Overwatch,880,24390,27445,https://static-cdn.jtvnw.net/ttv-boxart/Overwatch-272x380.jpg,https://static-cdn.jtvnw.net/ttv-logoart/Overwatch-240x144.jpg
